In [1]:
import nltk
from nltk.tokenize import sent_tokenize, word_tokenize
nltk.download()
from nltk import ngrams, FreqDist
import re
import string
from multiprocessing import Pool
from nltk.corpus import brown
from nltk.stem import PorterStemmer
import pandas as pd

showing info https://raw.githubusercontent.com/nltk/nltk_data/gh-pages/index.xml


In [2]:
#Creating a wordlist with all the words for this script
wordlist = [w for w in nltk.corpus.words.words('en') if w.islower()]
print('Entire word list has {} words.'.format(len(set(wordlist))))
print(wordlist[:100])
#list of the alphabet to iterate through later
alphabet = list(string.ascii_lowercase)

Entire word list has 210687 words.
['a', 'aa', 'aal', 'aalii', 'aam', 'aardvark', 'aardwolf', 'aba', 'abac', 'abaca', 'abacate', 'abacay', 'abacinate', 'abacination', 'abaciscus', 'abacist', 'aback', 'abactinal', 'abactinally', 'abaction', 'abactor', 'abaculus', 'abacus', 'abaff', 'abaft', 'abaisance', 'abaiser', 'abaissed', 'abalienate', 'abalienation', 'abalone', 'abampere', 'abandon', 'abandonable', 'abandoned', 'abandonedly', 'abandonee', 'abandoner', 'abandonment', 'abaptiston', 'abarthrosis', 'abarticular', 'abarticulation', 'abas', 'abase', 'abased', 'abasedly', 'abasedness', 'abasement', 'abaser', 'abash', 'abashed', 'abashedly', 'abashedness', 'abashless', 'abashlessly', 'abashment', 'abasia', 'abasic', 'abask', 'abastardize', 'abatable', 'abate', 'abatement', 'abater', 'abatis', 'abatised', 'abaton', 'abator', 'abattoir', 'abature', 'abave', 'abaxial', 'abaxile', 'abaze', 'abb', 'abbacomes', 'abbacy', 'abbas', 'abbasi', 'abbassi', 'abbatial', 'abbatical', 'abbess', 'abbey', '

In [3]:
#Creating a stemmed word list to eliminate derivatives of words
porter = PorterStemmer()
stem_list = []
for word in wordlist:
    temp = porter.stem(word)
    stem_list.append(temp)
stem_list = list(dict.fromkeys(stem_list))
print('Entire stemmed word list has {} words.'.format(len(stem_list)))
print(stem_list[:100])

Entire stemmed word list has 156802 words.
['a', 'aa', 'aal', 'aalii', 'aam', 'aardvark', 'aardwolf', 'aba', 'abac', 'abaca', 'abacay', 'abacin', 'abaciscu', 'abacist', 'aback', 'abactin', 'abact', 'abactor', 'abaculu', 'abacu', 'abaff', 'abaft', 'abais', 'abaiss', 'abalien', 'abalon', 'abamper', 'abandon', 'abandonedli', 'abandone', 'abaptiston', 'abarthrosi', 'abarticular', 'abarticul', 'abas', 'abasedli', 'abased', 'abash', 'abashedli', 'abashed', 'abashless', 'abashlessli', 'abasia', 'abask', 'abastard', 'abat', 'abati', 'abatis', 'abaton', 'abattoir', 'abatur', 'abav', 'abaxi', 'abaxil', 'abaz', 'abb', 'abbacom', 'abbaci', 'abba', 'abbasi', 'abbassi', 'abbati', 'abbat', 'abbess', 'abbey', 'abbeysted', 'abbot', 'abbotci', 'abbotnulliu', 'abbotship', 'abbrevi', 'abbreviatori', 'abbreviatur', 'abcoulomb', 'abdal', 'abdat', 'abdest', 'abdic', 'abdit', 'abditori', 'abdomen', 'abdomin', 'abdominalian', 'abdominoanterior', 'abdominocardiac', 'abdominocentesi', 'abdominocyst', 'abdominoge

In [4]:
def createWordList(letter):
    '''
    function to create the list of all the words that begin with that letter
    '''
    my_regex = '^' + re.escape(letter)
    root_word = [w for w in wordlist if re.search(my_regex, w, re.IGNORECASE)]
    
    letter_list = []

    for w in range(len(root_word)):
        letter_list.append(root_word[w])
        
    return letter_list

In [5]:
def getRoots(total_list, j):
    '''
    function to find the best root (most off-shoot words) for the given number of letters, along with the number of off-shoots
    ex. If you say 4, how many words start with 'anti', how many start with 'alph', the root with the most words will be returned
    '''
    f = 0
    a_list = []
    n = 0
    m = 0
    winner_root = None

    for i in range(len(total_list)):
        last_word = total_list[i-1]
        word = total_list[i]
        try:
            if last_word[:j] == word[:j]:
                n = n + 1
                if a_list[-1] == last_word:
                    continue
                else:
                    a_list.append(last_word)
                    if a_list[-1] == word:
                        continue
                    else:
                        a_list.append(word)
                        if n > m:
                            m = n
                            winner_root = last_word[:j]
            else:
                if n > m:
                    n = n + 1
                    m = n
                    winner_root = last_word[:j]
                    n = 0
                else:
                    n = 0
        except:
            a_list.append(last_word)
            a_list.append(word)
            if word[:j] == total_list[i+1][:5]:
                a_list.append(total_list[i+1])
            f = f+1

    return winner_root, m

In [6]:
def bestRoots(rootNumber):
    '''
    input: number for how many letters in root
    output: the best root with the most off-shoot words for every letter, along with the count of how many off-shoot words
    '''
    root = []
    for i in range(len(alphabet)):
        root.append(getRoots(createWordList(alphabet[i]), rootNumber))
        #print(getRoots(createWordList(alphabet[i]), rootNumber))
    
    df = pd.DataFrame(root)
    df.columns = ['Root' + str(rootNumber), 'Count' + str(rootNumber)]
    df.rename(index = {0: 'a', 1: 'b', 2: 'c', 3: 'd', 4: 'e', 5: 'f', 6: 'g', 7: 'h', 8: 'i', 9: 'j', 10: 'k', 11: 'l', 12: 'm', 13: 'n', 14: 'o', 15: 'p', 16: 'q', 17: 'r', 18: 's', 19: 't', 20: 'u', 21: 'v', 22: 'w', 23: 'x', 24: 'y', 25: 'z'}, inplace=True)        
        
    return df

In [7]:
#example - getting the best root for 'q' obviously yields 'qu', and we learn it has 1067 words that start with 'qu'
print(getRoots(createWordList('q'), 2))
print(getRoots(createWordList('a'), 2)[0])
print(bestRoots(2))

('qu', 1067)
an
  Root2  Count2
a    an    3151
b    be    2063
c    co    6357
d    di    3471
e    en    1611
f    fo    1433
g    ga    1209
h    he    2297
i    in    5237
j    ja     339
k    ki     395
l    la    1553
m    ma    2505
n    no    3481
o    ov    2195
p    pr    5943
q    qu    1067
r    re    5371
s    su    4153
t    tr    3047
u    un   14483
v    ve     983
w    wa     921
x    xe     103
y    ya     171
z    zo     309


In [8]:
#df = pd.DataFrame({'root':[getRoots(createWordList('a'), 2)[0], getRoots(createWordList('b'), 2)[0]]}, index = ['a', 'b'])
df = bestRoots(1)
i=2
while True:
    df = pd.concat([df, bestRoots(i)], axis=1, join = 'inner')
    i = i + 1
    if i > 5:
        break
               
print(df)

  Root1  Count1 Root2  Count2 Root3  Count3 Root4  Count4  Root5  Count5
a     a   14537    an    3151   ant    1621  anti    1077  anthr     173
b     b    9675    be    2063   bra     459  back     151  brach     109
c     c   17405    co    6357   con    1895  coun     511  count     495
d     d    9945    di    3471   dis    1687  disc     337  disco     179
e     e    7817    en    1611   epi     567  elec     333  elect     331
f     f    6381    fo    1433   for     885  fore     467  fores      99
g     g    5843    ga    1209   gra     545  gast     199  gastr     181
h     h    7889    he    2297   hyp    1033  hype     485  hyper     483
i     i    8303    in    5237   int    1799  inte    1323  inter    1181
j     j    1157    ja     339   jac      75  jack      41  justi      27
k     k    1735    ki     395   kin     145  kera      67  kerat      49
l     l    5211    la    1553   lit     237  lith     143  litho     113
m     m   10709    ma    2505   mis     829  mono  

In [15]:
yellow = []
for word in wordlist:
    if word.startswith('yello'):
        yellow.append(word)
print(yellow)
print(len(yellow))

['yelloch', 'yellow', 'yellowammer', 'yellowback', 'yellowbelly', 'yellowberry', 'yellowbill', 'yellowbird', 'yellowcrown', 'yellowcup', 'yellowfin', 'yellowfish', 'yellowhammer', 'yellowhead', 'yellowing', 'yellowish', 'yellowishness', 'yellowlegs', 'yellowly', 'yellowness', 'yellowroot', 'yellowrump', 'yellows', 'yellowseed', 'yellowshank', 'yellowshanks', 'yellowshins', 'yellowtail', 'yellowthorn', 'yellowthroat', 'yellowtop', 'yellowware', 'yellowweed', 'yellowwood', 'yellowwort', 'yellowy']
36


In [8]:
#best four letter roots for all 26 letters
bestRoots(4)

('anti', 1077)
('back', 151)
('coun', 511)
('disc', 337)
('elec', 333)
('fore', 467)
('gast', 199)
('hype', 485)
('inte', 1323)
('jack', 41)
('kera', 67)
('lith', 143)
('mono', 529)
('nonc', 391)
('over', 2043)
('poly', 699)
('quin', 221)
('reco', 299)
('supe', 999)
('tran', 529)
('unde', 1333)
('vent', 105)
('wood', 101)
('xant', 79)
('yell', 39)
('zygo', 61)


In [12]:
#best three letter roots for all 26 letters
bestRoots(3)

('ant', 1621)
('bra', 459)
('con', 1895)
('dis', 1687)
('epi', 567)
('for', 885)
('gra', 545)
('hyp', 1033)
('int', 1799)
('jac', 75)
('kin', 145)
('lit', 237)
('mis', 829)
('non', 2787)
('ove', 2055)
('pre', 3017)
('qua', 477)
('rec', 757)
('sub', 1607)
('tri', 1195)
('und', 1899)
('ver', 383)
('whi', 321)
('xan', 79)
('you', 49)
('zoo', 213)


In [22]:
def wordsWithThatRoot(root):
    root_list = []
    for word in wordlist:
        if word.startswith(root):
            root_list.append(word)
        else:
            continue

    return root_list

In [23]:
brown_list = set(brown.words())
'accommodate' in brown_list

True

In [24]:
def countWithThatRoot(root):
    root_list = []
    for word in wordlist:
        if word.startswith(root):
            root_list.append(word)
        else:
            continue

    return len(root_list)

In [25]:
print(wordsWithThatRoot('accom'))

['accombin', 'accommod', 'accommodatingli', 'accompani', 'accompaniment', 'accompanist', 'accompanyist', 'accomplet', 'accomplic', 'accompliceship', 'accompl', 'accomplish', 'accomplisht', 'accompt']


In [26]:
def getLetterBreakdown(word):
    n = 1
    total = 0
    number = []
    results = []
    word_list = list(word)
    for i in range(len(word_list)):
        short = word[:n]
        each = wordsWithThatRoot(short)
        number = countWithThatRoot(short)
        results.append(each)
        n = n + 1
        total = total + number
        if number < 100:
            print(short + ' - ' + str(number) + " "+ str(', '.join(each)))
        else:
            print(short + ' - ' + str(number))
    print("The total number of possible words for " + word + ":" + str(total))

In [28]:
def getLetterCount(word):
    n = 1
    total = 0
    number = []
    results = []
    word_list = list(word)
    for i in range(len(word_list)):
        short = word[:n]
        number = countWithThatRoot(short)
        n = n + 1
        total = total + number

    return total

In [29]:
getLetterBreakdown("cherry")

c - 13001
ch - 2074
che - 298
cher - 18 chercock, cherem, cherimoya, cherish, cherishingli, chernozem, cheroot, cherri, cherryblossom, cherrylik, chersones, chert, cherti, cherub, cherubim, cherubin, chervil, chervonet
cherr - 3 cherri, cherryblossom, cherrylik
cherry - 2 cherryblossom, cherrylik
The total number of possible words for cherry:15396


In [30]:
getLetterCount("acanthopterygian")

11864

In [32]:
def bestWord():
    best_word = None
    for i in range(len(wordlist[:3000])):
        last_word = wordlist[i-1]
        current_word = wordlist[i]
        try:
            if getLetterCount(current_word) > getLetterCount(best_word):
                best_word = current_word
                print(best_word)
            else:
                continue
        except:
            best_word = current_word

    return best_word

In [ ]:
p = Pool(6)
print(p.map(bestWord()))

aa
aal
aalii
aardvark
aba
abac
abaca
abacay
abacinate
abaciscus
abashed
abashedly
abashless
abashlessly
abdominal
abdominalian
abdominoanterior
abdominocardiac
abdominohysterectomy
abstract
abstracted
abstractedly
abstraction
abstractionism
abstractionist
acacatechin
academe
academial
academian
academician
acanaceous
acanth
acantha
acanthaceous
acanthin
acanthion
acanthocarpous
acanthocephalan
acanthopteran
acanthopterygian
acetabular
acetabuliferous
acetabuliform
acetoacetanilide
acetonaphthone
acetonurometer
acetonylacetone
acetylacetonates
acetylbenzene
acetylenediurein
acetylphenylhydrazine


In [64]:
fdist = FreqDist(i.lower() for i in brown.words())

In [65]:
fdist["caama"]

0

In [23]:
def mostRare(word_list):
    best_word = None
    for i in range(len(word_list)):
        last_word = word_list[i-1]
        current_word = word_list[i]
        try:
            if fdist[current_word] < fdist[last_word]:
                best_word = current_word
            else:
                continue
        except:
            best_word = current_word
    
    print(best_word)

In [24]:
a_words = [w for w in nltk.corpus.words.words('en') if w.islower() and w.startswith('a')]

In [25]:
mostRare(wordlist)

zythum


In [78]:
for word in brown_list:
    if fdist[word] == 1:
        print(word)

sighs
lob
bloke
dynamited
mai
sorcery
embossed
ultra-high-speed
hander
allure
hel
halogens
mailed-fist-in-velvet-glove
resigning
franks-in-buns
dual-ladder
vied
abbreviations
hard-bitten
petered
self-exile
spaniel's
peer-group
drowsily
bulkheads
granulocytic
once-popular
conventionality
infirmity
interpretable
erudition
well-wedged
ornate
abscissa
$172,400
261
pre-cooled
one-sixteenth
self-enclosed
fellow-men
radiates
shipboard
gassy
infernally
classifying
perfectibility
ground-level
shifty
party-line
discard
fiction-writer's
$754
$7,000,000
rippled
missile's
lovelies
non-competitive
consumer's
incompleteness
holdovers
semi-inflated
$634,517,000
mechanic's
soft-shoe
condemnatory
kraft
long-far
trenches
meaty
totted
963
textbooks
greenness
state-administered
amulet
cometary
crocked
schoolgirl
institution-wide
dauphin
2.5%
nonsystematic
entreat
re-explore
skylight
unstuffy
1,257,700
sun-browned
break-away
7-room
rankles
super-experiment
megakaryocytic
enchant
postage-prepaid
coolheaded
c

canceling
$300,000
cross-purposes
bibliographical
4,369
lifeboats
rhymes
thoroughness
rateable
cooperates
fluff
6934
half-cocked
procreativity
offences
dissented
stultifying
palm-studded
quivers
cowboy's
hurdled
electrodynamics
underpinning
paternalistic
thicken
euphoric
group's
pig-infested
bluebook
eye-machine
influenza-pneumonia
telepathically
interposed
21-year-old
flautist
unredeemed
pectoral-front
nibs'
nabbed
balling
emasculation
incriminating
expend
phloem
gratifyingly
pinhead
sin-ned
corpuscular-radiation
gainer
ward-heelers
permeate
behynde
flood-ravaged
peccavi
woolgather
$12
wallpapers
straightens
slow-bouncing
hinders
$29,000
slick-headed
hemispherical
saute
electives
poignantly
underarm
agitate
gainful
27-in.
4,122,354
diphosphopyridine
self-reliance
antiquarians
creativity-oriented
chuckles
seamanship
sweethearts
cottonseed
free-buying
minimizes
gritty-eyed
nut-like
subroutine
elaborates
semi-heights
pleasance
chain-reaction
enny
contaminate
re-emergence
cosy
satin-cover

accommodation
pagoda
ropers
liquidations
seedless
needlessly
pier-table
antiredeposition
sociable
make-believe
quits
demonstrators
vacuolated
limelight
one-act
$581,000
116
baited
optimizing
proliferated
polonaise
electorate
vesicular
pesticides
mincing
rough-and-tumble
sling
hymens
scopes
concocted
calumny
worryin'
critics'
awaye
redheaded
spear-throwing
chimes
ablated
face-lifting
wil
cohorts
300th
recreating
strainin'
summons
shelled
judgements
besieging
sneezing
hoi-polloi
sub-chiefdom
abduction
semitropical
dollar-and-cents
disgust
city-bred
wainscoted
crisis-to-crisis
supposes
non-success
bleat
solar-electromagnetic-
newly-married
242%
103
costumed
pyocanea
restating
incontrovertible
referent
verandas
1598/9
wash-outs
beholds
prayerbooks
smokes
renunciations
hankered
funnier
unadorned
tyrants
transpirating
chargeable
50-year-old
spaniel-like
best-tempered
valiantly
non-dealer
nil
$50,400,000
humped
pastel-like
edifying
paraphrasing
thumb-sucking
scarcely-tapped
reconnaissanace
si

1068
enraptured
neonatal
scathing
tamper
valiant
thet's
officiate
matriarch
prodding
check-out
clap
recommence
gossamer
1-hp
catlike
microscopical
value-orientations
killers
blood-flow
erythroid
somethin
clout
medicinal
many-much
bullying
hour's
clove
plainclothes
gallants
sleeper's
cornucopia
creeps
$8.50
trenchermen
asymmetry
pipers
near-synonyms
stoutly
rubberized
counterchallenge
teammates'
prospering
inviolable
toil
presuppose
unendurable
do-good
wherefores
lads
wage-setter
commandeered
missile-type
demoralization
tagua
surface-analyzer
1,119
castigated
chaise
warmish
yuse
freshened
depressants
marketwise
bun
$31,179,816
choir's
heroically
boxer
queued
unreasoning
37,679
nipped
olive-flushed
tartly
1515
mum
bodyweight
arboreal
beauteous
attired
gymnast
triphenylarsine
misdemeanants
mural
sycophants
pro-tem
peptide
din
half-dressed
abjectly
fritters
well-ruled
self-judging
small-arms
snob-clannish
analyticity
sippers
big-town
matsyendra
emigrating
deployment
malposed
bandoleers
fel

$14,000
complimenting
delenda
inarticulate
mycobacteria
sassing
brewing
executing
tangy
cartoonists
castigates
darting
parisology
lobule
mustachioed
weirdy
stringing
reimburse
finalist
level's
retching
buckwheat
unaccountable
1597
iron-shod
relishing
more-than-average
bigots
commences
unsheathe
chromatogram
distributor's
eight-and-a-half-foot
187-mile
flounders
lawman's
cornbread
stained-glass
hove
impersonated
ringers
pacifistic
frog-eating
conceptually
parachutes
chicanery
helpfulness
ravines
rusting
normals
razor-sharp
repeater
shinbone
unrepentant
rebuked
widowhood
all-purpose
slaughtering
boy-meets-girl
taps
parsympathetic
fatherly
froth
delicate-beyond-description
camellias
brownish
low-speed
apostates
gratis
milliamperes/cell
patriots
impoundments
valeur
maht
wus
deras
questioningly
unreasonably
573
denominated
rock'n'roll
obsessions
non-itemized
beef's
daylight's
communicator's
pre-conditions
halftime
dissenters
sharpness
26-2
obsoleting
togs
5-4
croupier
plated
alizarin
bruite

...

pellagra
$2.30/mbf
cosmical
orphans
wormy
befuddles
consecration
fauna
stomach-belly
seaquake
6:30
wrings
encroach
6124
7,500
walled
cut-glass
198
hearing-aid
cleat
dawns
1802
fainted
inter-relationships
remounting
unmotivated
8861
reproof
esters
tycoon
cavernous
highland
conjugation
dragon
ipso
tho'
nise
thorns
curtain-raiser
flop
indomitable
nonobservant
raisin
fast-spreading
crossings
pyorrhea
pathogenesis
briskness
goodies
400-kc.
49%
re-adopt
felony
wonder-working
$3,825
aspired
octopus
foulest
infuriate
re-incorporated
misconstructions
lopped
biter
normative
overreaches
cross-fertilized
noes
357
aches
cherubim
presentness
trouble-shooter
imagnation
all-round
thermopile
nationalize
trobles
surgical
near-blind
much-copied
double-
flog
longshot
pre-assault
subjectivists
wrappin'
offensively
gangplank
sunt
reprobating
safe-conduct
imbibe
wavy-haired
out-of-door
fur-piece
sago
273
hardwoods
1733
geographers
iridium
autobiographic
depose
questionaire
harmlessly
afield
movie-to-be
d'hot

flannels
sugared
labor's
rhetoricians
minter
eye-strain
harboring
outcrops
vacationland
antislavery
guerilla
addressees
guileless
felicitous
cobblestone
tuneful
encroached
caterpillars
gesticulated
barn-burner's
surfeited
double-talk
hand-hewn
lobster
deflated
refracted
undaunted
eight-week
non-color
24-degrees
registrar
writer-turned-painter
dinghy
wooden-leg
$2,300
half-clad
601
daunted
pupated
light-flared
extravaganzas
sub-group
haberdasheries
moistening
allurement
finisher
172nd
outplayed
invalids
left-justified
weirs
thrombosis
blowfish
drudgery
steroid-induced
unnerving
pre-1960
job-seekers
$140
earth-touching
forsakes
495
industrially
fail-safe
$4,177.37
too-simple-to-be-true
fresh-ground
leaches
0.85
scuffle
a-raising
60-66
nation-building
collared
exhaustingly
librettists
sanguineum
survivalist
troughs
fall's
rattler
unworn
gumming
coarsely
idol-worship
interference-like
dining-room
12-1/2
sellin'
2,418
obstinate
polystyrene
displacing
recently-passed
revolted
reground
telegr

shooters
blooded
flag-stick
canals
honeymooners
flourishing
redeeming
choosy
hooliganism
startled-horse
hay-wagon
anti-liquor
astuteness
anti
knock-down
yearningly
feudalistic
exact-size
equidistantly
intradepartmental
$66,000
mutters
thiocyanate-perchlorate-fluoro
hodgepodge
unfrocking
retriever
avocation
droppings
boastfully
game-management
semi-private
kamikaze
legacies
$8,313,514
capricious
tunefulness
rasped
blonde-headed
continence
shorelines
calluses
comas
angling
warningly
half-standard
multi-family
campaign's
spurned
mill-pond
crackled
uneducated
reappearing
droves
coachwork
g{t}-axis
d-c
illustrators
homecomings
specters
shrilling
writer's
more'n
blustery
anchoring
weed
blue-collar
predomination
erroneously
0.5-mv./m.
hookups
kilowatts
rubbery
rights-of-way
platform-controller
silky
pressure-sensing
brockle
8-mm
chimera-chasing
once-a-month
sneer
irruptions
trustingly
nullity
cynically
spinneret
empathy
naturopath
regenerating
egregiously
$2,557,111
overindulged
instructors
h

anomaly
unalterable
northerly
vopos
couches
poems-in-drawing-and-type
store-front
luring
flagellation
scaring
offersey
sputter
emanated
mis-reading
catecholamines
self-mastery
insertion
swami
starboard
weatherbeaten
awkwardness
three-judge
burne
convenient-type
open-face
boatsmen
untenanted
union-industry
team-mate
adjudging
tetrahalides
312
garishness
monolithic
morsels
bull-roaring
lucidity
legation
adipic
socked
imaginatively
12-oz.
surcease
substitutionary
good-humoredly
ritiuality
ousted
humanize
introjected
unformed
$109
turnoff
princess-in-a-carriage
recompence
excavations
diaphragms
reharmonization
bomber's
quipping
third-rate
forgitful
scurrilous
horse-playing
379,900
demineralization
diorah
congregated
$20,000,000,000
subversives
re-enacting
last-round
point-blank
force-rate
bumble-bee
self-perceived
traineeships
hecatomb
dyspeptic
jotted
cheerfulness
a-wing
deferment
switchblade
pimpled
faze
$102,285,000
swampy
draftee
growers'
speakin'
carloads
tightest-fitting
bookish
glob

glum
boaters
enciphered
misted
tailin's
pinging
toweling
succinct
deathly
somersaulting
orgy
scorcher
stodgy
knoe
printmaking
outposts
conspiratorial
clattery
wronged
paleoexplosion
all-inclusive
healthiest
towboats
schoolwork
four-thirty
dials
bugler
closed-circuit
scrapbook
fourteen-nation
infantrymen
deftness
shute
magnifies
consanguineous
oscillator
tributes
play-off
open-handed
bicarbonate
capacitors
blinkers
subsist
salesman's
1637
opportune
braids
grassroots-fueled
subtler
thickening
rotundity
reconciliation
885
bridgework
autocratic
massifs
mean-square
rail-mobile
parts-suppliers
quickness
restock
kwashiorkor
sub-zero
meekest
ballerina
unexamined
unidirectional
gaging
worktable
barrel-vaulted
quirk
franker
imbued
galled
tokenish
localize
swearinge
symbolic-sounding
architects'
seismographs
sarcolemmal
foamed-in-place
doves
diligently
whole-heartedly
konga
$55,000
clown's
wrest
self-image
lyophilized
secreted
codified
sawing
unjustified
navigate
non-stop
ideational
6a
16.7
dunk
